In [25]:
import numpy as np
import matplotlib.pyplot as plt
from pde import CartesianGrid, ScalarField, PDE, FieldCollection

In [ ]:
def Keqcalc(T):
  R = 8.314  # J / mol·K

  #CO2; CO; CH4; H2O; H2 = 00000
  G_const = np.array([
    [-393.4685065, -0.003212871, 1.53E-07, 9.97E-10, -3.31E-13],
    [-110.7347984, -0.086473798, -9.63E-06, 8.56E-09, -2.11E-12],
    [-68.77179183, 0.038329957, 9.22E-05, -5.46E-08, 1.24E-11],
    [-240.6171143, 0.035116356, 2.02E-05, -9.33E-09, 1.78E-12]
  ])
  Gf = np.sum(G_const * np.array([1, T, T**2, T**3, T**4]), axis=1)
  Keqsmr = np.exp(-(Gf[1] - Gf[2] - Gf[3]) * 1000 / (R * T))
  Keqwgs = np.exp(-(Gf[0] - Gf[1] - Gf[3]) * 1000 / (R * T))
  return np.array([Keqsmr, Keqwgs])

In [29]:
# Constants
P = 3 #atm
T = 1000  # K
u0 = 79.64 * 273 / T * P / 1 / 60 #cm^3/sec
A = np.pi * (3.5/10)**2 #cm^2. I assumed a 7 mm diameter
v0 = u0 / A / 100 #m/sec

Keq = np.array([101325**2 * np.exp(-26830/T + 30.114), np.exp(4400/T - 4.036)])
#Keq = Keqcalc(T)

# Stoichiometry Matrix
nu = np.array([[-1, 0, 0], [-1, -1, 0], [1, -1, 0], [0, 1, 0], [3, 1, 1], [0, 0, 0]])

#reaction constants
gasconstant = 8.314 #J / mol·K
Easmr = 165.740 #kJ/mol FROM THE PAPER MbodjiCommengeFalk
Asmr = 1.68 * 10**8 #FROM THE PAPER MbodjiCommengeFalk
Eawgs = 89.23 # kJ/mol
Awgs = 9.90 * 10**3

L = 0.3 #m

#adjust this
Am = 0.03/10**-1 #15 cm2/130 mg
Mc = 1 #13 mg
PR = 1 #50%
Psc = 1 #7mm check L = 6.09 cm
Ku = (Am * Mc * PR) / (Psc * L)

I = 9; #Curent in Amp
F = 96485; #faradays constant in C/mol

#grid for length of reactor
N = 100
grid = CartesianGrid([[0, L]],[N])

#Boundry condition for ch4
# Cch40  = ych40 = 0.3
# bcs = [{"value": 0.3}, {"value": 0.6}, {"value": 0}, 
#       {"value": 0}, {"value": 0}, {"value": 0.1}]
# ics = FieldCollection([ScalarField(grid, 0) for i in range(6)])
ics = ScalarField(grid, 0)

#UPDATE THESE EQUATIONS LIKE IN THE PAPER MbodjiCommengeFalk 
# might need to split these up
def rxneq(C):
    R = np.zeros(3)
    R[0] = Ku * Asmr * np.exp(-Easmr * 1000 / (gasconstant * T)) * (C[0] * C[1] - C[2] * C[3]**3 / Keq[0])  #Keq IS IN UNITS Pa^2
    R[1] = Ku * Awgs * np.exp(-Eawgs * 1000/ (gasconstant * T))*(C[2] * C[1] - C[3] * C[4] / Keq[1])
    R[2] = I / (2 * F)
    return R


def vcalc():
    v = 0.5
    return v

# https://py-pde.readthedocs.io/en/latest/examples_gallery/simple_pdes/pde_brusselator_expression.html

eq = PDE(
    {
        "Cch4": f"divergence(Cch4)" 
        #np.dot({nu[0]}, rxneq([Cch4,0,0,0,0]))"

    }
)



results = eq.solve(ics, t_range=1, dt=0.001)
results.plot()


ValueError: Incompatible shapes of input array